# Motivation
论文原文：[《Wide & Deep Learning for Recommender Systems》](https://arxiv.org/abs/1606.07792)

参考学习地址：[详解 Wide & Deep 结构背后的动机](https://zhuanlan.zhihu.com/p/53361519)

Wide&Deep核心思想是结合**线性模型的记忆能力（Memorization）**和**DNN模型的泛化能力（Generalization）**。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/wide&deep.png" width = "1000" height = "300" alt="Wide&Deep网络结构" align=center />

## Memorization
CTR预估问题中存在大规模Sparse特征，普遍做法是“特征交叉+LR”。其中Memorization通过一系列人工的特征交叉（outer-product）来构造非线性特征，捕捉sparse特征之间的高阶相关性。例如，特征1——专业: {计算机、人文、其他}，特征2——下载过音乐《消愁》:{是、否}，这两个特征one-hot后的特征维度分别为3维与2维，对应的交叉结果是特征3——专业☓下载过音乐《消愁》: {计算机∧是，计算机∧否，人文∧是，人文∧否，其他∧是，其他∧否}。

典型代表是LR模型，使用大量Sparse特征（Dense也通常会被分桶离散化）和交叉特征作为输入。优点是模型可解释性高，实现快速高效，特征重要度易于区分；缺点是需要人工设计，过拟合概率高（极端输入所有特征交叉，无异于记住所有样本），无法捕捉训练中未出现过的特征对（对应的交叉权重始终为0）。

对应Wide部分是广义线性模型
$$
\begin{align}
y=\mathbf{w}^T[\mathbf{x},\phi(\mathbf{x})]+\mathbf{b}
\end{align}
$$
其中$\mathbf{x}$和$\phi(\mathbf{x})$分别表示原始特征与交叉特征。


## Generalization
Sparse特征学习低维的Dense Embeddings来捕获特征相关性，类似NLP词向量，也是基于相关性之间的传递。优点是较少的人工参与，缺失样本的交叉组合也有泛化性。在推荐系统中，当user-item matrix非常稀疏时，例如有和独特爱好的users以及很小众的items，NN很难为users和items学习到有效的embedding。这种情况下，大部分user-item应该是没有关联的，但dense embedding 的方法还是可以得到对所有 user-item pair 的非零预测，因此导致 over-generalize并推荐不怎么相关的物品。

对应Deep部分是前馈神经网络，注意Deep中仅有黄点（也即没有人工交叉的特征）
$$
\begin{align}
\mathbf{a}^{l+1}=f(\mathbf{W}^l\mathbf{a}^l+\mathbf{b}^l)
\end{align}
$$
其中$\mathbf{W}^l$，$\mathbf{a}^l$和$\mathbf{b}^l$分别表示第l层的权重，激活值和偏置，$f$是激活函数。


## Loss
采用联合损失计算
$$
\begin{align}
p(y=1|x)=\sigma(\mathbf{w}_{wide}^T[\mathbf{x},\phi(\mathbf{x})]+\mathbf{w}_{deep}^T\mathbf{a}^{l_f}+\mathbf{b})
\end{align}
$$
其中$\mathbf{a}^{l_f}$代表NN最后一层激活值。


## 常见向量计算术语
假设向量$a=(1,0,0)$和$b=(0,1,0)$
* 内积（FM）表示向量相乘，1\*n矩阵 \* n\*1 矩阵
  $$
  \begin{align}
  a\cdot b=a^Tb = a_1b_1+a_2b_2+a_3b_3=0
  \end{align}
  $$

* 外积（Outer Product）
  指张量外积或向量外积，两个向量的外积形成一个矩阵，矩阵的每个元素$(i,j)=a_i*b_j$，n\*1矩阵 \* 1\*n 矩阵
  $$
  \begin{align}
  a\otimes b=ab^T=\begin{pmatrix} a_1b_1 & a_1b_2 & a_1b_3 \\ a_2b_1 & a_2b_2 & a_2b_3 \\a_3b_1 & a_3b_2 & a_3b_3 \\ \end{pmatrix}_{3 \times 3}=\begin{pmatrix} 0 & 1 & 0 \\ 0 & 0 & 0 \\0 & 0 & 0 \\ \end{pmatrix}_{3 \times 3}
  \end{align}
  $$

* 叉乘（Cross Product）
  严格属于三维欧几里得空间，输入两个三维向量输出一个新的三维向量。叉乘公式为
  $$
  \begin{align}
  a\times b=(a_2b_3-a_3b_2,a_3b_1-a_1b_3,a_1b_2-a_2b_1)=(0,0,1)
  \end{align}
  $$

* 哈达玛积（NFM，xDeepFM，AFM）
  同维向量间的逐元素交互
  $$
  \begin{align}
  a\odot b = (a_1b_1, a_2b_2, a_3b_3)=(0,0,0)
  \end{align}
  $$

内积就是哈达玛积再求和